# Testing Pretrained ASR models

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

drive  sample_data


In [3]:
%cd 'drive/My Drive/Speech Recognition project/'

/content/drive/My Drive/Speech Recognition project


In [4]:

# Downloading the Caytu whosper models
!  pip install git+https://github.com/sudoping01/whosper.git

  Cloning https://github.com/sudoping01/whosper.git to /tmp/pip-req-build-z86gdr8e
  Running command git clone --filter=blob:none --quiet https://github.com/sudoping01/whosper.git /tmp/pip-req-build-z86gdr8e
  Resolved https://github.com/sudoping01/whosper.git to commit 9922c752d782d1eafc5eb1aad57a2324e4892479
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision v4.49.0) to /tmp/pip-install-zs6sgwsd/transformers_c14ac1e9933e420cb62adbe830d63b37
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-zs6sgwsd/transformers_c14ac1e9933e420cb62adbe830d63b37
  Running command git checkout -q dcc2b5e633d7a2cb35ad9f1a2466b7258a91dade
  Resolved https://github.com/huggingface/transformers.git to commit dcc2b5e633d7a2cb35ad9f1a2466b7258a91dade
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... do

In [5]:
import pandas as pd
import numpy as np
import torch
import torchaudio
from whosper import WhosperTranscriber


In [6]:
df = pd.read_csv("Wavtext_dataset2.csv")
df.head()

,text,path_wav
0,Nu ngi indee woon jiwu ji sapoN,recordings/200702-231930_wol_8ec_elicit/200702...
1,Nu ngi tollu dagana,recordings/200702-231930_wol_8ec_elicit/200702...
2,Nu ngi wàkkirlu ci yàlla,recordings/200702-231930_wol_8ec_elicit/200702...
3,Nu ñow nag def ko ci kër gi,recordings/200702-231930_wol_8ec_elicit/200702...
4,Nuggaayu néeg bee ko fi téye,recordings/200702-231930_wol_8ec_elicit/200702...


In [7]:
transcriber = WhosperTranscriber(model_id = "CAYTU/whosper-large-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [8]:
# Test on the the first row of the dataset
result = transcriber.transcribe_audio(df.loc[0, "path_wav"])

print(result['text'])

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Nu nu indee woon njiw wu ci sapoŋ


In [9]:
print(df.loc[0, "text"])

Nu ngi indee woon jiwu ji sapoN


In [10]:
result2 = transcriber.transcribe_audio(df.loc[1, "path_wav"])

print(result2)

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': 'Nu ngi tollu dagana'}


In [1]:
print(df.loc[1, "text"])

NameError: name 'df' is not defined

In [12]:
!pip install librosa soundfile sounddevice


In [13]:
result2 = transcriber.transcribe_audio(df.loc[100, "path_wav"])

print(f"predicted transcription: {result2['text']}")
print(f"actual transcription: {df.loc[100, 'text']}")


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


predicted transcription: naka lañuy dugg ci biir dëkk bi ñu daje ay waxambaane yu takku yu jëmoon tool
actual transcription: Nakka lañu dugg ci biir dëkk bi ñu dajeeg ay waxambaane yu takku yu jëmoon tool


In [14]:
import librosa
import IPython.display as ipd

# Load audio
audio_path = df.loc[100, "path_wav"]
audio, sr = librosa.load(audio_path, sr=16000)

# Play it
ipd.Audio(audio, rate=sr)

# Text To Speech

In [17]:
%cd 'drive/My Drive/Speech Recognition project/'


/content/drive/My Drive/Speech Recognition project


In [14]:
!pip install -U datasets
!pip install fsspec==2023.9.2

In [15]:
import torch
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor, SpeechT5HifiGan
from datasets import load_dataset
from IPython.display import Audio, display

def load_speech_model(checkpoint="bilalfaye/speecht5_tts-wolof-v0.2", vocoder_checkpoint="microsoft/speecht5_hifigan"):
    """ Load the SpeechT5 model, processor, and vocoder for text-to-speech. """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    processor = SpeechT5Processor.from_pretrained(checkpoint)
    model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint).to(device)
    vocoder = SpeechT5HifiGan.from_pretrained(vocoder_checkpoint).to(device)

    return processor, model, vocoder, device

# Load the model
processor, model, vocoder, device = load_speech_model()

# Load speaker embeddings (pretrained from CMU Arctic dataset)
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

def generate_speech_from_text(text, speaker_embedding=speaker_embedding, processor=processor, model=model, vocoder=vocoder):
    """ Generates speech from input text using SpeechT5 and HiFi-GAN vocoder. """

    inputs = processor(text=text, return_tensors="pt", padding=True, truncation=True, max_length=model.config.max_text_positions)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    speech = model.generate(
        inputs["input_ids"],
        speaker_embeddings=speaker_embedding.to(model.device),
        vocoder=vocoder,
        num_beams=7,
        temperature=0.6,
        no_repeat_ngram_size=3,
        repetition_penalty=1.5,
    )

    speech = speech.detach().cpu().numpy()
    display(Audio(speech, rate=16000))

# Example usage French
text = "Bonjour, bienvenue dans le modèle de synthèse vocale Wolof et Français."
generate_speech_from_text(text)

# Example usage Wolof
text = "ñu ne ñoom ñooy nattukaay satélite yi"
generate_speech_from_text(text)


RuntimeError: Dataset scripts are no longer supported, but found cmu-arctic-xvectors.py

In [13]:
df.head(20)

,text,path_wav
0,Nu ngi indee woon jiwu ji sapoN,recordings/200702-231930_wol_8ec_elicit/200702...
1,Nu ngi tollu dagana,recordings/200702-231930_wol_8ec_elicit/200702...
2,Nu ngi wàkkirlu ci yàlla,recordings/200702-231930_wol_8ec_elicit/200702...
3,Nu ñow nag def ko ci kër gi,recordings/200702-231930_wol_8ec_elicit/200702...
4,Nuggaayu néeg bee ko fi téye,recordings/200702-231930_wol_8ec_elicit/200702...
5,Nun dungeen nu woo,recordings/200702-231930_wol_8ec_elicit/200702...
6,Nun dunu dem,recordings/200702-231930_wol_8ec_elicit/200702...
7,Nun la yokkute gi di sonal,recordings/200702-231930_wol_8ec_elicit/200702...
8,Nunnil sëllu wi ci ndey ji,recordings/200702-231930_wol_8ec_elicit/200702...
9,Nuru na ku xiifoon dañu ko jox añ bi mu ne ko ...,recordings/200702-231930_wol_8ec_elicit/200702...


In [14]:
generate_speech_from_text(df.loc[16, "text"])

NameError: name 'generate_speech_from_text' is not defined

In [ ]:
df.loc[16, "text"]

# Fine Tuning

## Recording app